# Test Data Generation: Base-Truth Tables Demonstration Notebook

**Affiliation**: *Kwantum Edu Analytics*. **Last Modified**: *2/14/2023*.

This OEA test data generation notebook illustrates use of the OEA_py and base_test_data_gen_py python classes for creating and developing base-truth table test data in stage1.

Use the main function outlined in the base_test_data_gen_py class notebook ```gen_base_tables(numstudents, numschools, numenroll, hed_numcourses, ed_level)``` to create test data for the base-truth tables.

Refer to the class notebook for understanding/editing functionality, options, and generation processes.

In [1]:
%run OEA_py

StatementMeta(, 13, -1, Finished, Available)

2023-02-14 17:58:33,138 - OEA - INFO - Now using workspace: dev
2023-02-14 17:58:33,139 - OEA - INFO - OEA initialized.


In [2]:
# set the workspace (this determines where in the data lake you'll be writing to and reading from).
# You can work in 'dev', 'prod', or a sandbox with any name you choose.
# For example, Sam the developer can create a 'sam' workspace and expect to find his datasets in the data lake under oea/sandboxes/sam
oea.set_workspace('dev')

StatementMeta(spark3p2medTDG, 13, 2, Finished, Available)

2023-02-14 17:58:34,036 - OEA - INFO - Now using workspace: dev


In [3]:
%run /base_test_data_gen_py

StatementMeta(, 13, -1, Finished, Available)

In [4]:
datagen = TestDataGen_BaseTables()

StatementMeta(spark3p2medTDG, 13, 4, Finished, Available)

In [5]:
# Read comments for generation-options.

education_level = 'hed' # NOTE: choose k12 or hed. k12 = generating K-12 test data, hed = generating Higher Education test data
num_students = 200 # NOTE: number of students to generate
num_schools = 3 # NOTE: number of schools to generate (k12 min allowed value: 4, hed max allowed value: 3)
hed_num_courses = 10 # NOTE: ONLY for hed. number of courses per school to generate (max allowed value: 21)
num_sections_enrolled_per_student = 5 # NOTE: number of classes each middle and high school student, or higher ed. student, is to be enrolled in (max allowed value: 21)

datagen.gen_base_tables(numstudents=num_students, numschools=num_schools, numenroll=num_sections_enrolled_per_student, hed_numcourses=hed_num_courses, ed_level=education_level)

StatementMeta(spark3p2medTDG, 13, 5, Finished, Available)

## Demo of How to Read-in and Check the Created Base-Truth Tables

Code-blocks are broken up into tests per each table created:
 1. **base_students**
 2. **base_schools**
 3. **base_courses**
 4. **base_sections**
 5. **base_enrollment**

In [6]:
# 1) base_students
dfStudents = oea.load_csv('stage1/Transactional/test_data/v0.1/base_students/snapshot_batch_data/*/*.csv', header=True)
print('Number of students generated (should be ' + str(num_students) + '):')
print(dfStudents.count())
display(dfStudents.limit(20))

StatementMeta(spark3p2medTDG, 13, 6, Finished, Available)

Number of students generated (should be 200):
200


SynapseWidget(Synapse.DataFrame, e0dc2ab4-9637-495b-86e8-62b7a95adf74)

In [7]:
# 2) base_schools
dfSchools = oea.load_csv('stage1/Transactional/test_data/v0.1/base_schools/snapshot_batch_data/*/*.csv', header=True)
display(dfSchools.limit(10))

StatementMeta(spark3p2medTDG, 13, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 30599b9b-c779-4fca-9526-537b16ae85db)

In [8]:
# 3) base_courses
dfCourses = oea.load_csv('stage1/Transactional/test_data/v0.1/base_courses/snapshot_batch_data/*/*.csv', header=True)
# if K-12 gen: 21 courses per high school, 24 courses per middle school, 6 courses per elementary school
if education_level == 'k12':
    dfCount = dfSchools.groupBy('SchoolType').count()
    temp = dfCount.filter(dfCount['SchoolType'] == 'High School')
    num_hs_gen = temp.collect()[0][1]
    temp = dfCount.filter(dfCount['SchoolType'] == 'Middle School')
    num_ms_gen = temp.collect()[0][1]
    temp = dfCount.filter(dfCount['SchoolType'] == 'Elementary School')
    num_es_gen = temp.collect()[0][1]
    total_courses = (num_hs_gen * 21) + (num_ms_gen * 24) + (num_es_gen * 6)
    print('There should be ' + str(total_courses) + ' K-12 courses generated.')
# if Higher Ed. gen: num_courses input * num_schools input
else:
    num_schools_gen = dfSchools.count()
    total_courses = num_schools * hed_num_courses
    print('There should be ' + str(total_courses) + ' Higher Ed. courses generated.')
print('Actual number of courses generated: ' + str(dfCourses.count()))
display(dfCourses)

StatementMeta(spark3p2medTDG, 13, 8, Finished, Available)

There should be 30 Higher Ed. courses generated.
Actual number of courses generated: 30


SynapseWidget(Synapse.DataFrame, 62ebcaf4-d81f-405a-8931-8da8ea1b3d84)

In [9]:
# 4) base_sections
dfSections = oea.load_csv('stage1/Transactional/test_data/v0.1/base_sections/snapshot_batch_data/*/*.csv', header=True)
num_courses_gen = dfCourses.count()
# if K-12 gen: 1-2 sections per course per high/middle school, 2-4 sections for 6 courses per elementary school
if education_level == 'k12':
    min_sections_gen = num_courses_gen + (num_es_gen * 6)
    max_sections_gen = (num_courses_gen * 2) + ((num_es_gen * 6)*2)
    print('There should be between ' + str(min_sections_gen) + ' and ' + str(max_sections_gen) + ' K-12 sections generated.')
# if Higher Ed. gen: 1-2 sections per the number of courses generated
else:
    min_sections_gen = num_courses_gen 
    max_sections_gen = num_courses_gen*2
    print('There should be between ' + str(min_sections_gen) + ' and ' + str(max_sections_gen) + ' Higher Ed. sections generated.')
print('Actual number of sections generated: ' + str(dfSections.count()))
display(dfSections.limit(10))

StatementMeta(spark3p2medTDG, 13, 9, Finished, Available)

There should be between 30 and 60 Higher Ed. sections generated.
Actual number of sections generated: 48


SynapseWidget(Synapse.DataFrame, 3822423c-8062-4ffd-99cf-535365aa8fba)

In [10]:
# 5) base_enrollment
dfEnroll = oea.load_csv('stage1/Transactional/test_data/v0.1/base_enrollment/snapshot_batch_data/*/*.csv', header=True)
# if K-12 gen: 1 enrollment per student in elementary school, num_sections_enrolled_per_student per student in high/middle school
if education_level == 'k12':
    dfCount = dfStudents.groupBy('SchoolType').count()
    temp = dfCount.filter(dfCount['SchoolType'] == 'Elementary School')
    num_es_students_gen = temp.collect()[0][1]
    num_hms_students_gen = num_students - num_es_students_gen
    total_enrollments = (num_hms_students_gen * num_sections_enrolled_per_student) + num_es_students_gen
    print('There should be ' + str(total_enrollments) + ' K-12 enrollments generated.')
# if Higher Ed. gen: num_sections_enrolled_per_student per student
else:
    total_enrollments = num_students * num_sections_enrolled_per_student
    print('There should be ' + str(total_enrollments) + ' Higher Ed. enrollments generated.')
print('Actual number of enrollments generated: ' + str(dfEnroll.count()))
display(dfEnroll.limit(10))

StatementMeta(spark3p2medTDG, 13, 10, Finished, Available)

There should be 1000 Higher Ed. enrollments generated.
Actual number of enrollments generated: 1000


SynapseWidget(Synapse.DataFrame, 2e54c3bf-c4bc-480f-bde2-b3944c50e28f)